In [ ]:
proj_id = 'cmat-315920'
root_path = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph', 'Districts']:
    %run -i {code_path+f+'.py'}

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

self = Gerry(abbr='TX', level='cntyvtd', district_type='cd', agg_polygon=True, simplification=300, num_colors=10,
          election_filters=(
            "office='President' and race='general'",
            "office='USSen' and race='general'",
#                 "office like 'USRep%' and race='general'",
          ),
          refresh_tbl=refresh_tbl, refresh_all=refresh_all
         )
# self.get_data()

start = time.time()
self.MCMC(steps=49)
self.agg_plans()
self.stack_plans()
elapsed = time.time() - start
h, m = divmod(elapsed, 3600)
m, s = divmod(m, 60)
print(f'{int(h)}hrs {int(m)}min {s:.2f}sec elapsed')

random_seed (default=1) 


Get crosswalks TX 2020 tabblock   ... not necessary ... success
Get assignments TX 2020 tabblock  ... tabblock table exists ... success
Get shapes TX 2020 tabblock       ... tabblock table exists ... success
Get census TX 2020 tabblock       ... tabblock table exists ... success
Get elections TX 2020 tabblock    ... tabblock table exists ... success
Get votes_all TX 2020 tabblock    ... tabblock table exists ... success
Get votes_hl TX 2020 tabblock     ... tabblock table exists ... success
Get combined TX 2020 cntyvtd      ... cntyvtd table exists ... success
Get edges TX 2020 cntyvtd         ... cntyvtd table exists ... success
Get nodes TX 2020 cntyvtd         ... success
Get graph TX 2020 cntyvtd cd      ... gpickle exists ... success
Get cd TX 2020 cntyvtd cd         ... success
Post-processing plan_00 to make cmat-315920.redistricting_data.cook_plans_TX_2020_cntyvtd_cd_plan_00 ... joining ... aggregating census, votes_all, votes_hl, polygons with simplification 300 with colors ..

In [ ]:
import os, google, numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt, plotly.express as px
from google.cloud import aiplatform, bigquery
try:
    from google.cloud.bigquery_storage import BigQueryReadClient
except:
    os.system('pip install --upgrade google-cloud-bigquery-storage')
    from google.cloud.bigquery_storage import BigQueryReadClient

proj_id = 'cmat-315920'
root_path = '/home/jupyter'
cred, proj = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bqclient   = bigquery.Client(credentials=cred, project=proj)

district_type = 'cd'
num_steps = 10
tbl = f'{proj_id}.redistricting_data.cook_plans_TX_2020_cntyvtd_{district_type}_stacked'
df = (bqclient.query(f'select * from {tbl}').result().to_dataframe()
      .rename(columns={'geoid':district_type})
     )
geo = gpd.GeoSeries.from_wkt(df['polygon'], crs='EPSG:4326').simplify(0.05).buffer(0)
gdf = gpd.GeoDataFrame(df, geometry=geo)
gdf['color'] = gdf['color'].astype(str)


X = gdf.query(f'plan < {num_steps}')
fig = px.choropleth(X, geojson=X.geometry, locations=X.index, projection="mercator",
                    color="color", color_discrete_sequence=px.colors.qualitative.T10, basemap_visible=True,
                    hover_data=[district_type, 'total_pop', 'polsby_popper'],
                    animation_frame="plan", animation_group=X[district_type],
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.write_html(root_path + '/recomb.html')
fig.show()